In [2]:
d1 = {"python": 10, "java": 3, "c#": 8, "javascript": 15}
d2 = {"java": 10, "c++": 10, "c#": 4, "go": 9, "python": 6}
d3 = {"erlang": 5, "haskell": 2, "python": 1, "pascal": 1}

In [5]:
from collections import defaultdict

def dict_builder(*args):
    d = defaultdict(int)
    for arg in args:
        if isinstance(arg, dict):
            for k, v in arg.items():
                d[k] += v
        else:
            raise TypeError("Wrong type")
        
    return d
        
dict_builder(d1, d2, d3)


defaultdict(int,
            {'python': 17,
             'java': 13,
             'c#': 12,
             'javascript': 15,
             'c++': 10,
             'go': 9,
             'erlang': 5,
             'haskell': 2,
             'pascal': 1})

In [17]:
from collections import Counter

def count_dict(*args):
    c = Counter()
    for arg in args:
        c.update(arg)
    return c

count_dict(d1, d2, d3)

Counter({'python': 17,
         'javascript': 15,
         'java': 13,
         'c#': 12,
         'c++': 10,
         'go': 9,
         'erlang': 5,
         'haskell': 2,
         'pascal': 1})

In [20]:
eye_colors = ("amber", "blue", "brown", "gray", "green", "hazel", "red", "violet")

class Person:
    def __init__(self, eye_color):
        self.eye_color = eye_color

from random import seed, choices
seed(0)
persons = [Person(color) for color in choices(eye_colors[2:], k=50)]

In [53]:
d = defaultdict(int)
for person in persons:
    d[person.eye_color] += 1

d

defaultdict(int,
            {'violet': 12,
             'red': 10,
             'green': 8,
             'gray': 10,
             'hazel': 7,
             'brown': 3})

In [72]:
c = Counter((person.eye_color for person in persons))
c.update(eye_colors)
c.subtract(eye_colors)
c

Counter({'violet': 12,
         'red': 10,
         'gray': 10,
         'green': 8,
         'hazel': 7,
         'brown': 3,
         'amber': 0,
         'blue': 0})

In [66]:
base = Counter(("amber", "blue", "brown", "gray", "green", "hazel", "red", "violet"))
base

Counter({'amber': 1,
         'blue': 1,
         'brown': 1,
         'gray': 1,
         'green': 1,
         'hazel': 1,
         'red': 1,
         'violet': 1})

In [67]:
c.subtract(base)

In [69]:
c

Counter({'violet': 11,
         'red': 9,
         'gray': 9,
         'green': 7,
         'hazel': 6,
         'brown': 2,
         'amber': -1,
         'blue': -1})

In [70]:
c

Counter({'violet': 11,
         'red': 9,
         'gray': 9,
         'green': 7,
         'hazel': 6,
         'brown': 2,
         'amber': -1,
         'blue': -1})

In [23]:
import json
from collections import ChainMap

with open("common.json") as f:
    common = json.load(f)

with open("dev.json") as f:
    dev = json.load(f)

with open("prod.json") as f:
    prod = json.load(f)

dev

{'data': {'input_root': '/dev/path/inputs',
  'output_root': '/dev/path/outputs',
  'numerics': {'type': 'float'},
  'operators': {'add': '__add__'}},
 'database': {'user': 'test', 'pwd': 'test'},
 'logs': {'level': 'trace',
  'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s %(filename)s %(funcName)s %(message)s'}}

In [21]:
def apply_chainmap(common=common, dev=dev):
    for k in dev:
        if isinstance(dev[k], dict):
            dev[k] = apply_chainmap(dev=dev[k])
            dev[k] = ChainMap(dev[k])
    
    # for k in common:
    #     if isinstance(common[k], dict):
    #         common[k] = apply_chainmap(common=common[k])
    #         common[k] = ChainMap(common[k])



    # for k in common:
    #     if isinstance(common[k], dict):

    return dev

apply_chainmap()

{'data': ChainMap({'input_root': '/dev/path/inputs', 'output_root': '/dev/path/outputs', 'numerics': ChainMap({'type': 'float'}), 'operators': ChainMap({'add': '__add__'})}),
 'database': ChainMap({'user': 'test', 'pwd': 'test'}),
 'logs': ChainMap({'level': 'trace', 'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s %(filename)s %(funcName)s %(message)s'})}

{'data': {'input_root': '/dev/path/inputs',
  'output_root': '/dev/path/outputs',
  'numerics': {'type': 'float'},
  'operators': {'add': '__add__'}},
 'database': {'user': 'test', 'pwd': 'test'},
 'logs': {'level': 'trace',
  'format': '%(asctime)s: %(levelname)s: %(clientip)s %(user)s %(filename)s %(funcName)s %(message)s'}}

In [352]:
import json
from collections import ChainMap

with open("common.json") as f:
    common = json.load(f)

with open("dev.json") as f:
    dev = json.load(f)

with open("prod.json") as f:
    prod = json.load(f)


# d1 = {"a": 1, "b": 2, "c": {"hello": "world", "mulen": "rouge"}}
# d2 = {"b": 3, "d": 5, "c": {"hello": "jason", "big": "tree"}}

def update_dict(dev, common, isRecursive=False):
    for k in dev:
        if isinstance(dev[k], dict) and common.get(k) is not None:
            dev[k] = update_dict(dev[k], common[k])
            dev[k] = ChainMap(dev[k], common[k])
            dev[k].update({key: value for key, value in common[k].items() if key not in dev[k]})
    if not isRecursive:
        dev = ChainMap(dev, common)
    return dev

dev = update_dict(dev, common)
dev.update((k, v) for k, v in common.items() if k not in dev)


In [354]:
pprint(dev)

ChainMap({'data': ChainMap({'input_root': '/dev/path/inputs',
                            'numerics': ChainMap({'type': 'float'},
                                                 {'precision': 6,
                                                  'type': 'Decimal'}),
                            'operators': {'add': '__add__'},
                            'output_root': '/dev/path/outputs'},
                           {'input_root': '/default/path/inputs',
                            'numerics': {'precision': 6, 'type': 'Decimal'},
                            'output_root': '/default/path/outputs'}),
          'database': ChainMap({'pwd': 'test', 'user': 'test'},
                               {'db_name': 'deepdive',
                                'port': 5432,
                                'schema': 'public'}),
          'logs': ChainMap({'format': '%(asctime)s: %(levelname)s: '
                                      '%(clientip)s %(user)s %(filename)s '
                              

In [344]:
import json
from collections import ChainMap

with open("common.json") as f:
    common = json.load(f)

with open("dev.json") as f:
    dev = json.load(f)

with open("prod.json") as f:
    prod = json.load(f)

def chain_recursive(d1, d2):
    chain = ChainMap(d1, d2)
    for k, v in d1.items():
        if isinstance(v, dict) and k in d2:
            chain[k] = chain_recursive(d1[k], d2[k])

    return chain

dev = chain_recursive(dev, common)
pprint(dev)

ChainMap({'data': ChainMap({'input_root': '/dev/path/inputs',
                            'numerics': ChainMap({'type': 'float'},
                                                 {'precision': 6,
                                                  'type': 'Decimal'}),
                            'operators': {'add': '__add__'},
                            'output_root': '/dev/path/outputs'},
                           {'input_root': '/default/path/inputs',
                            'numerics': {'precision': 6, 'type': 'Decimal'},
                            'output_root': '/default/path/outputs'}),
          'database': ChainMap({'pwd': 'test', 'user': 'test'},
                               {'db_name': 'deepdive',
                                'port': 5432,
                                'schema': 'public'}),
          'logs': ChainMap({'format': '%(asctime)s: %(levelname)s: '
                                      '%(clientip)s %(user)s %(filename)s '
                              

In [339]:
from copy import deepcopy

In [340]:
a = deepcopy(dev)

In [345]:
b = deepcopy(dev)

In [349]:
b = 1